In [110]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import pandas as pd

In [141]:
def pad_images(image, filter_size):
    p = int((filter_size - 1)/2)
    image = np.pad(array=image, pad_width=p, mode='constant', constant_values=0)
    return image

def create_dataset(gray, red, green, blue, f_s):
    X = []
    y = []
    p_gray = pad_images(gray, f_s)
    p_red = pad_images(red, f_s)
    p_green = pad_images(green, f_s)
    p_blue = pad_images(blue, f_s)
    
    for i in range(0, len(p_gray)-(f_s-1)):
        for j in range(0, len(p_gray)-(f_s-1)):
            X.append(list(p_gray[i:i+f_s,j:j+f_s].flatten()))
            y.append([p_red[i:i+f_s,j:j+f_s].flatten()[int(f_s*f_s/2)], 
                      p_green[i:i+f_s,j:j+f_s].flatten()[int(f_s*f_s/2)], 
                      p_blue[i:i+f_s,j:j+f_s].flatten()[int(f_s*f_s/2)]])
    
    return X, y
    
def get_images(directory, f_s):
    exts = ["jpg", "jpeg", "png"]
    print("Opening directory {}".format(directory))
    for root, dirs, files in os.walk(directory):
        if root:
            X = []
            y = []
            file_name = []
            for f in files:
                if f.split(".")[1] in exts:
                    image = cv2.imread(os.path.join(root, f))
                    image = cv2.resize(image, (20,20), interpolation = cv2.INTER_AREA)
                    # gray image
                    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                    # red, green, blue components
                    red, green, blue = image[:,:,2], image[:,:,1], image[:,:,0]
                    m_X, m_y = create_dataset(gray, red, green, blue, 3)
                
                    X.append(m_X)
                    y.append(m_y)
                    file_name.append(f)
    return X, y, file_name
                

In [144]:
directory = "./Images/"
X, y, files = get_images(directory)
print("X: ", len(X[0]))
print("y: ", len(y[0]))
print("Filename: ", files)

Opening directory ./Images/
X:  400
y:  400
Filename:  ['scene8.jpeg', 'scene10.jpeg', 'scene9.jpeg', 'scene2.jpeg', 'scene1.jpeg', 'scene5.jpeg', 'scene3.jpeg', 'scene7.jpeg', 'scene6.jpeg']


In [103]:
a = np.random.randint(5, size=(3,3))
b = np.random.randint(5, size=(3,3))
c = np.random.randint(5, size=(3,3))
d = np.random.randint(5, size=(3,3))

In [104]:
print("a:\n {} \nb:\n {} \nc:\n {} \nd:\n {} \n".format(a,b,c,d))

a:
 [[1 1 1]
 [2 4 4]
 [3 3 3]] 
b:
 [[3 0 3]
 [2 0 4]
 [0 2 4]] 
c:
 [[4 1 1]
 [0 4 0]
 [1 2 3]] 
d:
 [[1 2 3]
 [0 0 4]
 [1 4 4]] 



In [105]:
a = np.pad(array=a, pad_width=1, mode='constant', constant_values=0)
b = np.pad(array=b, pad_width=1, mode='constant', constant_values=0)
c = np.pad(array=c, pad_width=1, mode='constant', constant_values=0)
d = np.pad(array=d, pad_width=1, mode='constant', constant_values=0)
print("a:\n {} \nb:\n {} \nc:\n {} \nd:\n {} \n".format(a,b,c,d))

a:
 [[0 0 0 0 0]
 [0 1 1 1 0]
 [0 2 4 4 0]
 [0 3 3 3 0]
 [0 0 0 0 0]] 
b:
 [[0 0 0 0 0]
 [0 3 0 3 0]
 [0 2 0 4 0]
 [0 0 2 4 0]
 [0 0 0 0 0]] 
c:
 [[0 0 0 0 0]
 [0 4 1 1 0]
 [0 0 4 0 0]
 [0 1 2 3 0]
 [0 0 0 0 0]] 
d:
 [[0 0 0 0 0]
 [0 1 2 3 0]
 [0 0 0 4 0]
 [0 1 4 4 0]
 [0 0 0 0 0]] 



In [109]:
X=[]
y = []
for i in range(0, len(a)-2):
    for j in range(0, len(a)-2):
        X.append(list(a[i:i+3,j:j+3].flatten()))
        y.append([b[i:i+3,j:j+3].flatten()[4],
                 c[i:i+3,j:j+3].flatten()[4],
                 d[i:i+3,j:j+3].flatten()[4]])
        
print("X: ", X)
print("y: ", y)

X:  [[0, 0, 0, 0, 1, 1, 0, 2, 4], [0, 0, 0, 1, 1, 1, 2, 4, 4], [0, 0, 0, 1, 1, 0, 4, 4, 0], [0, 1, 1, 0, 2, 4, 0, 3, 3], [1, 1, 1, 2, 4, 4, 3, 3, 3], [1, 1, 0, 4, 4, 0, 3, 3, 0], [0, 2, 4, 0, 3, 3, 0, 0, 0], [2, 4, 4, 3, 3, 3, 0, 0, 0], [4, 4, 0, 3, 3, 0, 0, 0, 0]]
y:  [[3, 4, 1], [0, 1, 2], [3, 1, 3], [2, 0, 0], [0, 4, 0], [4, 0, 4], [0, 1, 1], [2, 2, 4], [4, 3, 4]]
